<a href="https://colab.research.google.com/github/Jaimemorillo/ShouldIwatchThisMovie/blob/master/memoria_preprocessing_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preprocesamiento

El objetivo de esta sección es el de ver que técnicas existen para tratar de reducir el vocabulario de las oraciones. Con esto conseguimos acelerar y mejorar el entrenamiento de los modelos, ya que en el input de lo que vamos a codificar ya va la información esencial y la que contiene mayor carga de significado.

Podemos encontrarnos palabras con errores ortográficos, palabras que no aportan significado, palabras con terminaciones diferentes pero el mismo valor...

Si conseguimos tratar todos estos puntos vamos a conseguir extraer las características que realmente importan de los textos.

In [78]:
!python -m spacy download es_core_news_sm

✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')


In [79]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [80]:
import pandas as pd
import numpy as np
import time

import nltk
from nltk.tokenize import WordPunctTokenizer

import string
import re

from tqdm.notebook import tqdm
tqdm.pandas()

nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...


/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nltk_data]    |   Package shakespeare is already up-to-date!
[nltk_data]    | Downloading package stopwords to /root/nltk_data...
[nltk_data]    |   Package stopwords is already up-to-date!
[nltk_data]    | Downloading package treebank to /root/nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package omw to /root/nltk_data...
[nltk_data]    |   Package omw is already up-to-date!
[nltk_data]    | Downloading package wordnet to /root/nltk_data...
[nltk_data]    |   Package wordnet is already up-to-date!
[nltk_data]    | Downl

True

In [81]:
dataover = pd.read_csv("gdrive/My Drive/TFG/tmdb_spanish_def.csv", sep='#',encoding='utf-8', lineterminator='\n')
taste = pd.read_csv("gdrive/My Drive/TFG/tmdb_spanish_Jaime_def_2.csv", sep='#', encoding='utf-8')

In [82]:
taste['id'] = taste['id'].astype(int)

data = taste.merge(dataover[['id','title','overview','genres','crew','cast']], left_on='id', right_on='id')
data = data[~pd.isna(data.overview)]

data = data.dropna(subset=['like'])
data['like'] = data['like'].astype(int)

data = data.drop_duplicates(subset=['id'])
data.reset_index(inplace=True,drop=True)

print(len(data))

print(data.like.value_counts(dropna=False))

1015
0    514
1    501
Name: like, dtype: int64


## Correción ortográfica

Uno de los problema al que nos enfrentamos cuando hablamos de textos es al de los errores ortográficos, muchas veces escribimos rápido y nos pasamos por alto acentos, confundimos letras por tener sonidos similares, intercalamos letras de más, tenemos letras en mayúsculas y minúsculas, juntamos palabras...

Algunos de ellos como las tildes tienen una solución sencilla que además suelen ser los errores más habituales. Nuestra base de datos de partida es editable por los usuarios y es un punto necesario para ser tratado.

Vamos a ver las funciones y técnicas que nos van a ayudar a resolver este apartado:

### Normalización

Vamos a quitar todas las tildes y llevar todas las letras a minúsculas, de tal manera que no pueda haber diferencias entre la palabra con tílde o sin ella o si tiene una letra en mayus. o en minus.
Además vamos a eliminar todos los signos de puntuación y espacios dobles.

In [83]:
example = data.overview[0]
example

'La princesa Leia, líder del movimiento rebelde que desea reinstaurar la República en la galaxia en los tiempos ominosos del Imperio, es capturada por las malévolas Fuerzas Imperiales, capitaneadas por el implacable Darth Vader, el sirviente más fiel del emperador. El intrépido Luke Skywalker, ayudado por Han Solo, capitán de la nave espacial "El Halcón Milenario", y los androides, R2D2 y C3PO, serán los encargados de luchar contra el enemigo y rescatar a la princesa para volver a instaurar la justicia en el seno de la Galaxia.'

In [84]:
def normalize(x):
  x = x.lower()
  replacements = (
      ("á", "a"),
      ("é", "e"),
      ("í", "i"),
      ("ó", "o"),
      ("ú", "u"),
      ("ñ", "n")
  )
  for a, b in replacements:
      x = x.replace(a, b)

  x = x.translate(str.maketrans('','',string.punctuation))
  x = x.translate(str.maketrans('','','ªº¡¿'))    
  x = x.replace("  ", " ")
  return x

In [85]:
normalize(example)

'la princesa leia lider del movimiento rebelde que desea reinstaurar la republica en la galaxia en los tiempos ominosos del imperio es capturada por las malevolas fuerzas imperiales capitaneadas por el implacable darth vader el sirviente mas fiel del emperador el intrepido luke skywalker ayudado por han solo capitan de la nave espacial el halcon milenario y los androides r2d2 y c3po seran los encargados de luchar contra el enemigo y rescatar a la princesa para volver a instaurar la justicia en el seno de la galaxia'

### Spell Checker

La alternativa sería usar un corrector ortográfico o bien implementado por tí o usando una librería de las que hay disponibles.

Los correctores ortográficos funcionan comparando cada una de las palabras contra las palabras del diccionario. Se calcula la distancia de **Levenshtein** respecto a las del diccionario y se selecciona aquella que tenga una distancia menor o 0 en el caso de ser una palabra correcta que está en el diccionario.

Esto computacionalmente es muy costoso de aplicar a todas las palabras de cada sinopsis por lo tanto se descarta en nuestra aplicación. Asumimos que el porcentaje de palabras mal escritas será muy bajo. Además puedes tener una palabra mal escrita que también exista en el diccionario y no estaríamos ganando nada o encontrarte un nombre propio y que te lo sustituya. También es complicado asignar cual es la palabra correcta si hay varias posibles.

https://github.com/barrust/pyspellchecker

https://en.wikipedia.org/wiki/Levenshtein_distance

In [86]:
%pip install pyspellchecker

In [87]:
from spellchecker import SpellChecker

spell = SpellChecker(language='es')

# find those words that may be misspelled
misspelled = spell.unknown(['Mi', 'rvaca', 'se', 'gllama', 'Paca'])

for word in misspelled:
  # Get the one `most likely` answer
  print(spell.correction(word))

  # Get a list of `likely` options
  print(spell.candidates(word))

llama
{'illama', 'llama'}
vaca
{'vaca'}


## StopWords (Palabras vacías)

Este es el nombre que reciben las palabras que carecen de significado, como artíículos, pronombres, preposiciones... que nos interesa filtrar antes del tratamiento de nuestras sinopsis. Son palabras que no van a aportar nada al modelo y únicamente funcionarían como ruido. No existe una lista definitiva de stopwords del español y cada problema puede tener la suya propia. En nuestro caso la nuestra la cargamos mediante un '.txt' que contiene aquellas que suelen ser consideradas palabras vacías en la mayor parte de las ocasiones y la aplicaremos la funcion de normalización vista antes.

Para sacar estas stopwords comparamos todas las palabras de nuestra frase contra las de la lista y si coincide con alguna la sacamos.

Podemos ver con un ejemplo como se nos hace practiamente indiferente en terminos de significado el leer una oración con o sin estas palabras vacías:

***La princesa Leia, líder del movimiento rebelde que desea reinstaurar la República en la galaxia en los tiempos ominosos del Imperio.***

***princesa leia lider movimiento rebelde desea reinstaurar republica galaxia tiempos ominosos imperio.***

Con esta técnica conseguimos reducir el tamaño de las sinopsis y del diccionario final en consecuencia.

https://es.wikipedia.org/wiki/Palabra_vac%C3%ADa


In [88]:
stop_words = pd.read_csv("gdrive/My Drive/TFG/stopwords-es.txt",header=None)
stop_words = stop_words[0].tolist()
stop_words = [normalize(word) for word in stop_words]

def delete_stop_words(x):
  words = x.split(' ')
  words = [word for word in words if word not in stop_words]
  x = str(' '.join(words))
  return x

In [89]:
example

'La princesa Leia, líder del movimiento rebelde que desea reinstaurar la República en la galaxia en los tiempos ominosos del Imperio, es capturada por las malévolas Fuerzas Imperiales, capitaneadas por el implacable Darth Vader, el sirviente más fiel del emperador. El intrépido Luke Skywalker, ayudado por Han Solo, capitán de la nave espacial "El Halcón Milenario", y los androides, R2D2 y C3PO, serán los encargados de luchar contra el enemigo y rescatar a la princesa para volver a instaurar la justicia en el seno de la Galaxia.'

In [90]:
delete_stop_words(normalize(example))

'princesa leia lider movimiento rebelde desea reinstaurar republica galaxia tiempos ominosos imperio capturada malevolas fuerzas imperiales capitaneadas implacable darth vader sirviente fiel emperador intrepido luke skywalker ayudado capitan nave espacial halcon milenario androides r2d2 c3po encargados luchar enemigo rescatar princesa volver instaurar justicia seno galaxia'

## Stemming

La forma en la que aparecen las palabras puede penalizar la frecuencia de estas, de tal manera que palabras de la misma familia semántica (caballo, caballería, caballos...) sean consideradas como totalmente diferentes por nuestros modelos, cuando realmente comparten significado, y acaban dotándolas de una importancia menor de la que podrían tener. Lo que tenemos que tratar es de llevar todas estas palabras a una forma única para que nuestros algoritmos les den un peso más próximo a la realidad.

La primera técnica que vamos a ver se conoce como Stemming. Esta técnica consiste en llevar una palabra a su raíz, esta raíz no tiene por qué pertenecer al diccionario, y se logra tras quitar los morfemas (prefijos y sufijos) de la palabra derivada. El resultado no es el más preciso ya que no se basa en ningún tipo de regla lingüística para quitar estos morfemas, pero en la mayoría de casos suele ser suficiente para mapear palabras de la misma familia a la misma raiz. Computacionalmente es mucho menos costoso y más simple que la alternativa que veremos a continuación, por lo tanto la técnica elegida para nuestro problema es esta, lo que nostros buscamos es velocidad en el procesado.

https://en.wikipedia.org/wiki/Stemming

Existen diferentes algoritmos para lograr el stemming, el más conocido es el algoritmo de Porter. Nosotros vamos a aplicar un stemmer implementado dentro de la librería NLTK llamado SnowballStemmer, el funcionanmiento e implementación es muy similar al de Porter pero es bastante más agresivo y preciso.

El algoritmo de Porter fue desarrolado por un informático britanico llamado Martin F. Porter en el año 1979 como parte de un proyecto mayor de recuperación de información. El idioma original es el inglés y la mayoría de la literatura y códigos los encontramos para este idioma, por lo tanto vamos a explicar la implementación del algoritmo para el inglés y no el español, para nuestra lengua habría que considerar cambios en los apartados de las condiciones de la implementación, pero a rasgos generales vamos a alcanzar a entender el algoritmo de igual manera. 

(Explicación detallada del algoritmo) https://vijinimallawaarachchi.com/2017/05/09/porter-stemming-algorithm/

(Explicación Snowball Español) https://snowballstem.org/algorithms/spanish/stemmer.html

Vamos a ver un ejemplo de como quedaría esta frase tras aplicar el stemming:

"Aquel es un **caballo** de la **caballería** militar, los otros **caballos** no"

"aquel es un **caball** de la **caball** milit , los otros **caball** no"



---


https://tartarus.org/martin/PorterStemmer/

https://www.geeksforgeeks.org/snowball-stemmer-nlp/


In [91]:
from nltk import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("spanish", ignore_stopwords=True)

def stem_sentence(sentence):
  stemmed_text = [stemmer.stem(word) for word in word_tokenize(sentence)]
  return " ".join(stemmed_text)

In [92]:
stem_sentence(delete_stop_words(normalize(example)))

'princes lei lid movimient rebeld dese reinstaur republ galaxi tiemp omin imperi captur malevol fuerz imperial capitan implac darth vad sirvient fiel emper intrep luk skywalk ayud capit nav espacial halcon milenari android r2d2 c3po encarg luch enemig rescat princes volv instaur justici sen galaxi'

In [93]:
stem_sentence("Aquel es un caballo de la caballería militar, los otros caballos no")

'aquel es un caball de la caball milit , los otros caball no'

## Lemmatization

Con esta técnica lo que logramos es llevar cada palabra a su lema, el lema es la forma básica de la palabra que encontraríamos como entrada en el diccionario (a diferencia del stemming), el lema representa a todas las demás palabras flexionadas (palabras alteradas mediante morfemas), por ejemplo, decir sería el lema de dije, pero también de diré o dijéramos. En este caso tras aplicar la técnica pasaríamos de tener tres palabras distintas en nuestro vocabulario a solo una, justo lo que queremos conseguir. Para aplicar este método necesitaríamos tener mapeadas todas las palabras con su respectivo lema y luego poder realizar esta transformación para todas las palabras de cada sinopsis, lo que requiere un tiempo de computación bastante elevado, además de que siempre van a quedar palabras residuales que no tengamos almacenada y se nos escapen. En nuestro caso hemos usado los lemmas de una librería llamada spacy que como podemos comprobar con el siguiente ejemplo no muestra resultados muy positivos, unicamente consigue llevarnos al lema 2 de las 3 palabras.

"Aquel es un **caballo** de la **caballería** militar, los otros **caballos** no"

"Aquel ser uno **caballo** de lo **caballería** militar , lo otro **caballo** no"

Lo ideal es aplicar ambos métodos para cubrir aquellos casos en los que el stemming puede fallar o aquellas palabras que no tengamos mapeadas en la lematización. Primero se aplicaría la Lemmatization y a continuación el Stemming. El resultado para el ejemplo sería este:

"aquel es un **caball** de la **caball** milit , los otros **caball** no"


https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

In [94]:
import spacy

nlp = spacy.load('es_core_news_sm')

def lemmatize_sentece(sentence):  
  doc = nlp(sentence)
  return ' '.join([word.lemma_ for word in doc])

In [95]:
lemmatize_sentece("Aquel es un caballo de la caballería militar, los otros caballos no")

'Aquel ser uno caballo de lo caballería militar , lo otro caballo no'

In [96]:
lemmatize_sentece(example)

'La princesa Leia , líder del movimiento rebelde que desear reinstaurar lo República en lo galaxia en lo tiempo ominoso del Imperio , ser capturar por los malévolo Fuerzas Imperiales , capitanear por el implacable Darth Vader , el sirviente más fiel del emperador . El intrépido Luke Skywalker , ayudar por Han Solo , capitán de lo nave espacial " El Halcón Milenario " , y lo androide , R2D2 y C3PO , ser lo encargar de luchar contra el enemigo y rescatar a lo princesa parir volver a instaurar lo justicia en el seno de lo Galaxia .'

In [97]:
stem_sentence(lemmatize_sentece("Aquel es un caballo de la caballería militar, los otros caballos no"))

'aquel ser uno caball de lo caball milit , lo otro caball no'

In [98]:
stem_sentence("Aquel es un caballo de la caballería militar, los otros caballos no")

'aquel es un caball de la caball milit , los otros caball no'

## Text Augmentation

Un problema al que nos enfrentamos al entrenar redes neuronales es a la cantidad de datos de las que disponemos, estos modelos demandan una cantidad enorme de datos y suelen lograr mejores resultados cuanto mayor sea esta, muchas veces no disponemos de tantos datos de entrenamiento como nos gustaría. Es aquí donde entra en juego el data augmentation (aumento de datos) en nuestro caso al tratarse de texto se le llama "Text Augmentation". Lo que tratamos con ello es de crear textos sintéticos que se parezcan a nuestros textos iniciales y que ayuden a afianzar conocimiento a nuestros modelos. Existen multiples métodos para conseguir este aumento de textos, vamos a repasar unos cuantos y veremos cual es la mejor elección. Hay que tener en cuenta que este text augmentation únicamente ha de aplicarse al train. 

https://towardsdatascience.com/data-augmentation-in-nlp-2801a34dfc28

https://neptune.ai/blog/data-augmentation-nlp

https://towardsdatascience.com/data-augmentation-library-for-text-9661736b13ff

https://github.com/makcedward/nlpaug

### Sinónimos

Este es el método más sencillo y seguramente el más usado, básicamente consiste en reemplazar n palabras del texto que no sean stopwords por sus respectivos sinónimos. Necesitas tener un diccionario de sinónimos almacenado o usar alguna librería que disponga de la implementación. La mayor limitación de esta técnica es que no todas las palabras tienen un sinónimo. Vamos a usar una librería llamada nlpaug para tratar un ejemplo, únicamente le indicamos que nos cambie una palabra. 


---


Un niño pequeño juega con una **pelota** redonda en el parque

Un niño pequeño juega con una **balón** redonda en el parque

In [99]:
%pip install nlpaug

In [100]:
%pip install torch>=1.6.0 transformers>=4.0.0

In [101]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as naf
from nlpaug.augmenter.word import WordAugmenter

from nlpaug.util import Action

In [102]:
aug = naw.SynonymAug(aug_src='wordnet', lang='spa', aug_min=5, aug_max=20, stopwords=stop_words)
augmented_text = aug.augment('Un niño pequeño juega con una pelota redonda en el parque', n=1)
print("Original:")
print('Un niño pequeño juega con una pelota redonda en el parque')
print("Augmented Text:")
print(augmented_text)

Original:
Un niño pequeño juega con una pelota redonda en el parque
Augmented Text:
Un alevín escaso juega con una pelota redonda en el parque para bebés


In [103]:
augmented_text = aug.augment(example, n=1)
print("Original:")
print(example)
print("Augmented Text:")
print(augmented_text)

Original:
La princesa Leia, líder del movimiento rebelde que desea reinstaurar la República en la galaxia en los tiempos ominosos del Imperio, es capturada por las malévolas Fuerzas Imperiales, capitaneadas por el implacable Darth Vader, el sirviente más fiel del emperador. El intrépido Luke Skywalker, ayudado por Han Solo, capitán de la nave espacial "El Halcón Milenario", y los androides, R2D2 y C3PO, serán los encargados de luchar contra el enemigo y rescatar a la princesa para volver a instaurar la justicia en el seno de la Galaxia.
Augmented Text:
La princesa Leia, líder del agitación rebelde que desea reinstaurar la República en la galaxia en los tiempos ominosos del Imperio, es capturada por las malévolas Fuerzas Imperiales, capitaneadas por el implacable Darth Vader, el sirviente más fiel del emperador. El intrépido Luke Skywalker, ayudado por Han Solo, capitán de la nave espacial " El Halcón Milenario ", y los androides, R2D2 y C3PO, serán los encargados de luchar contra el en

### Back translation

En esta técnica como su propio nombre indica lo fundamental es la traducción. Consiste en traduccir el texto inicial a un idioma cualquiera y volverlo a traduccir al idioma de origen. Con esto conseguimos algo similiar a el caso de los sinónimos pero nos aprovechamos de las tecnologías y APIs de traducción (Google Translate, Bing, Yandex) para dotar de mayor coherencia a estos nuevos textos. También pueden introducir variaciones en el orden de las palabras que enriquecen a nuestro modelo. El problema de este método viene de las limitaciones que imponen estas APIs y que en la mayoría de casos te hacen pasar por caja. Vamos a usar una libería llamada translators para usar estas APIs.

El texto generado a partir de nuestro ejemplo pasando por el japonés sería el siguiente:


---


Un niño pequeño juega con una pelota redonda en el parque

Niño jugando con bola redonda en el parque

https://github.com/uliontse/translators

https://arxiv.org/pdf/1511.06709.pdf

In [104]:
!pip install translators

In [105]:
import translators as ts

API = 'bing'

def translator_constructor(api):
    if api == 'google':
        return ts.google
    elif api == 'bing':
        return ts.bing
    elif api == 'baidu':
        return ts.baidu
    elif api == 'sogou':
        return ts.sogou
    elif api == 'youdao':
        return ts.youdao
    elif api == 'tencent':
        return ts.tencent
    elif api == 'alibaba':
        return ts.alibaba
    elif api == 'yandex':
        return ts.yandex
    else:
        raise NotImplementedError(f'{api} translator is not realised!')

translator = translator_constructor(API)

def translate(x, lang='en'):
    try:
        return translator(x, 'es', lang)
    except:
        print('KO')
        time.sleep(60)
        return translate(x, lang='en')

def back_translate(x, lang='en'):
    try:
        return translator(x, lang, 'es')
    except:
        print('KO')
        time.sleep(60)
        return translator(x, lang, 'es')

def translate_substitution(x, lang='en'):
  t = translate(x, lang)
  b = back_translate(t, lang)
  return b

In [106]:
lang='ru'
translate_substitution('Un niño pequeño juega con una pelota redonda en el parque', lang=lang)

'Niño juega con bola redonda en el parque'

In [107]:
example

'La princesa Leia, líder del movimiento rebelde que desea reinstaurar la República en la galaxia en los tiempos ominosos del Imperio, es capturada por las malévolas Fuerzas Imperiales, capitaneadas por el implacable Darth Vader, el sirviente más fiel del emperador. El intrépido Luke Skywalker, ayudado por Han Solo, capitán de la nave espacial "El Halcón Milenario", y los androides, R2D2 y C3PO, serán los encargados de luchar contra el enemigo y rescatar a la princesa para volver a instaurar la justicia en el seno de la Galaxia.'

In [108]:
translate_substitution(example, lang=lang)

'La princesa Leia, líder del movimiento rebelde, que quiere restaurar la República en la galaxia durante la época de la Omiosis del Imperio, es capturada por las fuerzas imperiales malvadas, capitaneadas por el implacable Darth Vader, el siervo más leal del emperador. El intrépido Luke Skywalker, respaldado por Han Solo, capitán de la nave espacial Halcón Milenario, y los androides, R2D2 y C3PO, serán responsables de luchar contra el enemigo y rescatar a la princesa para restaurar la justicia a la galaxia.'

### Word Embeddings

Como hemos visto en el aparatado de codificación de textos, los word embeddings mapeaban las palabras sobre un espacio geométrico, de tal manera que las palabras similares se encontraban próximas entre sí, existe una distancia entre ellas que podemos medir. Con esta nueva técnica lo que vamos a hacer es cambiar n palabras de nuestra sentencia por sus palabras más cercanas en el embedding. Lo normal en este caso es utilizar un embedding pre-entrenado dónde buscar esta palabra más cercana. Veamos un ejemplo usando la librería nlpaug.

https://towardsdatascience.com/data-augmentation-in-nlp-2801a34dfc28

---

Un niño pequeño juega con una pelota redonda en el parque

Un niño monstruoso juega con una pelota redonda en el parquizó

Como podemos ver debido a la alta cantidad de palabras con las que ha sido entrenado (no se comprueba sin son palabras bien escritas, de la lengua...) esta técnica realiza sustituciones un poco extrañas y difíciles de controlar.


In [109]:
# Embedding https://github.com/dccuchile/spanish-word-embeddings
# https://fasttext.cc/docs/en/crawl-vectors.html

model_dir = '/content/gdrive/MyDrive/TFG/'

aug = naw.WordEmbsAug(
    model_type='fasttext', model_path= model_dir + 'embeddings-m-model.vec', action="substitute", aug_p=0.2, stopwords=stop_words)

In [110]:
augmented_text = aug.augment('Un niño pequeño juega con una pelota redonda en el parque')
print("Original:")
print('Un niño pequeño juega con una pelota redonda en el parque')
print("Augmented Text:")
print(augmented_text)

Original:
Un niño pequeño juega con una pelota redonda en el parque
Augmented Text:
Un cachorrinho pequeño juega con una pelota redonda en el monumento


### Contextualized Word Embeddings
Para solucionar el problema de los word embbedings tracionales, que comentamos anteriormente en el apartado de encoding, han surgido una nueva arquitectura de modelos bidireccionales que son capaces de tener en cuenta el contexto de las palabras. Para el caso de text augmentation, lo que podemos hacer es dejar que estos modelos nos cambien las palabras. Dada una palabra de nuestra frase es capaz de elegir un reemplazo adecuado para ella basándose en las que tiene alrededor.
En concreto nostros vamos a dejar que BERT (uno de estos modelos bidireccionales) nos genere nuevos textos, para ello usaremos la misma librería que para los sinónimos "nlpaug".


---

Un niño pequeño juega con una pelota redonda en el parque

Un hombre pequeño juega con una pelota colorada en el parque

Nos devuelve algo con bastante más sentido que en el apartado anterior.



In [111]:
# Cased acepta mayusculas y tildes, uncased no (para el aug lo único que varía es la salida)
aug = naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action="substitute", aug_p=0.2, stopwords=stop_words)

In [112]:
augmented_text = aug.augment('Un niño pequeño juega con una pelota redonda en el parque')
print("Original:")
print('Un niño pequeño juega con una pelota redonda en el parque')
print("Augmented Text:")
print(augmented_text)

Original:
Un niño pequeño juega con una pelota redonda en el parque
Augmented Text:
Los niño estadounidense juega con una pelota redonda en el parque


### Generación de texto

Existen modelos capaz de generar texto a partir de una frase dada. Aprende el contexto de la oración y empieza a generar texto. Muchos de estos textos tienen una coherencia asombrosa. Los dos modelos más conocidos son GPT-2 y XLnet. El problema es que tienen un funcionamiento muchísimo mejor y más fiable en inglés que en español.

Nosotros los podemos usar de tal manera que dandole la introducción de una sinopsis dejar que el modelo nos la continue. Veamos un ejemplo con nlpaug.


---

Un niño pequeño juega con una pelota redonda en el parque

Un niño pequeño juega con una pelota redonda en el parque . m - el w , - k re for and ir o n S , s on - , is u ( w on all G t - , t

Como podemos ver, el texto que ha generado es totalmente aleatorio. Estos modelos son extremadamente grandes por lo que para usarlos necesitas una api externa, no puedes levantarlos en tu propia máquina.


In [113]:
# model_path: xlnet-base-cased, gpt2 or distilgpt2
aug = nas.ContextualWordEmbsForSentenceAug(model_path='xlnet-base-cased', temperature=1, top_k=None)

In [114]:
augmented_texts = aug.augment('The quick brown fox jumps over the lazy dog .')
print("Original:")
print('The quick brown fox jumps over the lazy dog .')
print("Augmented Texts:")
print(augmented_texts)

Original:
The quick brown fox jumps over the lazy dog .
Augmented Texts:
The quick brown fox jumps over the lazy dog . An infectious split seconds presents herself unto Scott.


### Propuesta

Aplicar sustitución por sinónimos, traducción y aplicar word embedding contextual. Todos ellos de manera aleatoria.

In [115]:
class CustomTranslateAug(WordAugmenter):
    def __init__(self, name='CustomWord_Aug', aug_min=1, aug_max=10, 
                 aug_p=0.3, stopwords=None, tokenizer=None, reverse_tokenizer=None, 
                 device='cpu', verbose=0, stopwords_regex=None):
        super(CustomTranslateAug, self).__init__(
            action='substitute', name=name, aug_min=aug_min, aug_max=aug_max, 
                 aug_p=aug_p, stopwords=stopwords, tokenizer=tokenizer, reverse_tokenizer=reverse_tokenizer, 
                 device=device, verbose=0, stopwords_regex=stopwords_regex)
        
        self.model = self.get_model()

    def substitute(self, data):
        lang = 'zh'
        return translate_substitution(data, lang=lang)
    
    def get_model(self):
        return None

In [116]:
aug = CustomTranslateAug()
aug.augment('Un niño pequeño juega con una pelota redonda en el parque')

'Un niño pequeño juega a la pelota en el parque'

In [117]:
aug.augment(example)

'La princesa Leia, líder del movimiento rebelde, quería restaurar la República a la Vía Láctea durante el ominoso período del Imperio, pero fue capturada por el malvado Ejército Imperial, dirigido por Darth Vader, el siervo más fiel del emperador. El intrépido Luke Skytrot, con la ayuda de Han Solo, el capitán de la nave espacial Halcón Milenario, y los robots R2D2 y C3PO, serán responsables de luchar contra el enemigo, salvar a la princesa y restablecer la justicia en la galaxia.'

In [229]:
# En la documentación de Bert recomiendan este modelo 
# BERT-Base, Multilingual (Not recommended, use Multilingual Cased instead)

"""
    :param str aug_src: Support 'wordnet' and 'ppdb' .
    :param str model_path: Path of dictionary. Mandatory field if using PPDB as data source
    :param str lang: Language of your text. Default value is 'eng'.
    :param float aug_p: Percentage of word will be augmented.
    :param int aug_min: Minimum number of word will be augmented.
    :param int aug_max: Maximum number of word will be augmented. If None is passed, number of augmentation is
        calculated via aup_p. If calculated result from aug_p is smaller than aug_max, will use calculated result from
        aug_p. Otherwise, using aug_max.
    :param list stopwords: List of words which will be skipped from augment operation.
    :param str stopwords_regex: Regular expression for matching words which will be skipped from augment operation.
    :param func tokenizer: Customize tokenization process
    :param func reverse_tokenizer: Customize reverse of tokenization process
    :param bool force_reload: Force reload model to memory when initialize the class.
        Default value is False and suggesting to keep it as False if performance is the consideration.
    :param str name: Name of this augmenter
    >>> import nlpaug.augmenter.word as naw
    >>> aug = naw.SynonymAug()
"""

aug = naf.Sequential([
    naw.SynonymAug(aug_src='wordnet', lang='spa', aug_p=0.5, aug_min=10, aug_max=40, stopwords=stop_words),        
    naw.SynonymAug(aug_src='wordnet', lang='spa', aug_p=0.5, aug_min=10, aug_max=40, stopwords=stop_words), 
    naw.SynonymAug(aug_src='wordnet', lang='spa', aug_p=0.5, aug_min=10, aug_max=40, stopwords=stop_words),           
    #CustomTranslateAug(),
    #naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action='substitute', aug_p=0.7, stopwords=stop_words),
    #naw.ContextualWordEmbsAug(model_path='bert-base-multilingual-cased', action='insert', aug_p=0.7, stopwords=stop_words)
])

In [230]:
def get_new_text(x):
  text = aug.augment(x) 
  text_2 = aug.augment(text) 
  return [text, text_2]

In [231]:
data['new_overviews'] = data.overview.progress_apply(lambda x: get_new_text(x))

In [232]:
data.head(5)

,id,like,title,overview,genres,crew,cast,new_overviews
0,11,1,La guerra de las galaxias. Episodio IV: Una nu...,"La princesa Leia, líder del movimiento rebelde...","[{'id': 12, 'name': 'Aventura'}, {'id': 28, 'n...","[{'credit_id': '52fe420dc3a36847f8000437', 'de...","[{'cast_id': 3, 'character': 'Luke Skywalker',...","[La princesa real verdadero Leia, líder del de..."
1,62,1,2001: Una odisea del espacio,"La historia de la humanidad, en diversos estad...","[{'id': 878, 'name': 'Ciencia ficción'}, {'id'...","[{'credit_id': '52fe4212c3a36847f8001a05', 'de...","[{'cast_id': 6, 'character': 'Dr. David Bowman...","[La bien comercial de la afección, en diversos..."
2,74,1,La guerra de los mundos,"Basada en una popular novela de HG Wells, narr...","[{'id': 12, 'name': 'Aventura'}, {'id': 53, 'n...","[{'credit_id': '52fe4213c3a36847f8002225', 'de...","[{'cast_id': 13, 'character': 'Ray Ferrier', '...","[Basada en una popular novela de HG Wells, nar..."
3,75,1,Mars attacks!,Parodia de los filmes de ciencia ficción de lo...,"[{'id': 35, 'name': 'Comedia'}, {'id': 14, 'na...","[{'credit_id': '52fe4214c3a36847f80022eb', 'de...","[{'cast_id': 6, 'character': 'President James ...",[Burla de los filmes de cognición invención de...
4,78,1,Blade Runner,"A principios del siglo XXI, la Tyrell Corporat...","[{'id': 878, 'name': 'Ciencia ficción'}, {'id'...","[{'credit_id': '52fe4214c3a36847f8002595', 'de...","[{'cast_id': 6, 'character': 'Rick Deckard', '...","[A principios del 100 XXI, la Tyrell Corporati..."


In [233]:
data.to_csv("gdrive/My Drive/TFG/overviews_final.csv", sep='#',encoding='utf-8', index=False)